In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
from collections import OrderedDict
from tqdm import tqdm
import datetime

cudnn.benchmark = True
plt.ion()   # interactive mode

In [2]:
num = 0
for filename in os.listdir('D:/data2'):
    entries = os.listdir('D:/data2/' + filename)
    # print(filename)
    num += len(entries)
print(num)

288


In [3]:
dataset = datasets.ImageFolder('D:/data', transform=transforms.Compose([
        # transforms.Resize((1280, 1024)),
        transforms.Resize((644, 644)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))
class_names = dataset.classes
len(class_names)

7

In [4]:
trainds, valds, testds = torch.utils.data.random_split(dataset, [0.4, 0.3, 0.3], generator=torch.Generator().manual_seed(15275))

dataloaders = {'train': torch.utils.data.DataLoader(trainds, batch_size=32,
                                             shuffle=True, num_workers=16), 
               'val': torch.utils.data.DataLoader(valds, batch_size=32,
                                             shuffle=True, num_workers=16), 
               'test': torch.utils.data.DataLoader(testds, batch_size=32,
                                             shuffle=True, num_workers=16)}

dataset_sizes = {'train': len(trainds), 
               'val': len(valds), 
               'test': len(testds)}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
len(class_names)

7

In [9]:
model_ft = models.resnet50(weights='IMAGENET1K_V1')
# Here the size of each output sample is set to 5.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
model_ft.fc = nn.Linear(2048, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [10]:
conf_matrix = torch.zeros(len(class_names), len(class_names))

def test_model(model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(testds)
    correct = 0.0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloaders['test']:
            X, y= X.to(device), y.to(device)
            outputs = model(X)
            _, preds = torch.max(outputs, 1)
            # print('===================')
            # print(preds)
            # print(y.data)
            correct += torch.sum(preds == y.data)
            # 更新混淆矩阵
            for t, p in zip(y, preds):
                conf_matrix[t, p] += 1

    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%")

In [11]:
# torch.save(model_ft.state_dict(), './pytorch-resnet50-91.9')

In [12]:
model_test = models.resnet50(weights='IMAGENET1K_V1')
# Here the size of each output sample is set to 5.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
model_test.fc = nn.Linear(2048, len(class_names))

model_test = model_test.to(device)

criterion = nn.CrossEntropyLoss()

In [13]:
model_test.load_state_dict(torch.load('./pytorch-resnet50-91.9'))

<All keys matched successfully>

In [14]:
test_model(model_test, criterion)
print(conf_matrix)

Test Error: 
 Accuracy: 91.9%
tensor([[ 8.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 0., 11.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  7.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  6.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  8.,  1.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  7.,  1.],
        [ 0.,  1.,  0.,  0.,  0.,  0., 32.]])
